In [ ]:
# ============================================================
# 🎮 CHECK GPU - PASTIKAN GPU P100 AKTIF!
# ============================================================
import subprocess

print("="*70)
print("🎮 CHECKING GPU AVAILABILITY")
print("="*70)

try:
    result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
    
    if result.returncode == 0:
        print("\n✅ GPU DETECTED!")
        print(result.stdout)
        
        # Check if it's P100
        if 'P100' in result.stdout:
            print("\n🎉 PERFECT! GPU P100 aktif!")
        elif 'T4' in result.stdout:
            print("\n✅ OK! GPU T4 aktif (sedikit lebih lambat dari P100)")
        else:
            print("\n✅ GPU aktif!")
    else:
        raise Exception("nvidia-smi failed")
        
except Exception as e:
    print("\n" + "="*70)
    print("⚠️  WARNING: GPU TIDAK TERDETEKSI!")
    print("="*70)
    print("\n📋 CARA AKTIFKAN GPU:")
    print("1. Klik icon Settings (⚙️) di kanan atas")
    print("2. Accelerator: Pilih 'GPU P100'")
    print("3. Klik 'Save'")
    print("4. Notebook akan restart, jalankan cell ini lagi")
    print("\n" + "="*70)
    print("\n⚠️  TRAINING TANPA GPU AKAN SANGAT LAMBAT!")
    print("   Estimasi: 40-50 jam vs 6-8 jam dengan GPU P100")
    print("="*70)

In [ ]:
# ============================================================
# 📦 EXTRACT DATASET - UNZIP BATIK ULTIMATE
# ============================================================
import os
import zipfile

print("="*70)
print("📦 EXTRACTING BATIK ULTIMATE DATASET")
print("="*70)

# Kaggle dataset path
dataset_zip = '/kaggle/input/batik-teist/batik_ultimate.zip'
extract_to = '/kaggle/working/batik_ultimate'

if os.path.exists(dataset_zip):
    print(f"\n✅ Found: {dataset_zip}")
    print(f"📂 Extracting to: {extract_to}")
    print("\n⏳ This will take 2-3 minutes...\n")
    
    # Extract
    with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
        zip_ref.extractall('/kaggle/working')
    
    print("\n✅ Extraction complete!")
    
    # Count files
    total_images = 0
    total_classes = 0
    
    if os.path.exists(extract_to):
        for root, dirs, files in os.walk(extract_to):
            if not dirs:  # Leaf directory
                img_files = [f for f in files if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
                if img_files:
                    total_images += len(img_files)
                    total_classes += 1
        
        print(f"\n📊 Dataset Statistics:")
        print(f"   Total classes: {total_classes}")
        print(f"   Total images: {total_images}")
        print(f"   Avg per class: ~{total_images // total_classes if total_classes > 0 else 0}")
        
        if total_images > 20000:
            print("\n🎉🎉🎉 PHENOMENAL DATASET!")
            print("   Expected accuracy: 95-99%+ 🚀🚀🚀")
    else:
        print("\n⚠️  Warning: Extract folder not found!")
else:
    print(f"\n❌ Dataset not found: {dataset_zip}")
    print("\n📋 CARA FIX:")
    print("1. Klik '+ Add Data' di kanan atas")
    print("2. Search: 'batik-teist' atau 'muhammadmaftuh'")
    print("3. Klik dataset 'Batik Teist'")
    print("4. Klik 'Add'")
    print("5. Run cell ini lagi")

print("\n" + "="*70)

In [ ]:
# ============================================================
# 📚 IMPORT LIBRARIES
# ============================================================
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import AdamW
import numpy as np
import matplotlib.pyplot as plt
import os

print("="*70)
print("📚 LIBRARIES LOADED")
print("="*70)
print(f"\n✅ TensorFlow version: {tf.__version__}")
print(f"✅ GPU available: {tf.config.list_physical_devices('GPU')}")
print("\n" + "="*70)

In [ ]:
# ============================================================
# ⚙️ CONFIGURATION
# ============================================================
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS_PHASE1 = 50
EPOCHS_PHASE2 = 50

print("\n" + "="*70)
print("🚀 TRAINING CONFIGURATION")
print("="*70)
print("\n⚡ Strategy:")
print("   • Dataset: 21,779 images (37 classes, ~470 per class)")
print("   • Model: EfficientNetB4 (ImageNet pretrained)")
print("   • Training: 2-Phase (50+50 epochs)")
print("   • Phase 1: Freeze base, LR 1e-3")
print("   • Phase 2: Fine-tune all, LR 1e-4")
print("   • Expected: 95-99% accuracy")
print("   • Time: 6-8 hours total")
print("="*70 + "\n")

In [ ]:
# ============================================================
# 🔄 DATA AUGMENTATION
# ============================================================
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=180,
    width_shift_range=0.5,
    height_shift_range=0.5,
    shear_range=0.5,
    zoom_range=[0.4, 2.0],
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.2, 2.0],
    channel_shift_range=80,
    fill_mode='reflect',
    validation_split=0.2
)

val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

print("✅ Augmentation MAKSIMAL activated!")
print("   Akan generate 5000+ variasi per class during training!")
print("   = Total ~185,000+ training samples per epoch! 🚀")

In [ ]:
# ============================================================
# 📂 LOAD DATASET
# ============================================================
dataset_path = '/kaggle/working/batik_ultimate'

print("="*70)
print("📂 LOADING DATASET")
print("="*70)
print(f"\n📂 Dataset path: {dataset_path}\n")

train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    dataset_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

num_classes = train_generator.num_classes
images_per_class = train_generator.samples // num_classes

print(f"\n✅ Data loaded successfully!")
print(f"   Jumlah kelas: {num_classes}")
print(f"   Training samples: {train_generator.samples}")
print(f"   Validation samples: {val_generator.samples}")
print(f"   Images per class: ~{images_per_class}")
print(f"   Class names: {list(train_generator.class_indices.keys())[:5]}... (showing first 5)")

if images_per_class > 400:
    print("\n🎉🎉🎉 PHENOMENAL! 400+ per class!")
    print("   Expected accuracy: 95-99%+ 🚀🚀🚀")

print("\n" + "="*70)

In [ ]:
# ============================================================
# 📊 CALCULATE TRAINING STEPS
# ============================================================
original_per_class = train_generator.samples // num_classes

# ADAPTIVE multiplier
if original_per_class > 300:
    multiplier = 2
    print("✅ Dataset is LARGE - using 2x multiplier")
else:
    multiplier = 3
    print("✅ Using 3x multiplier")

steps_per_epoch = (train_generator.samples // BATCH_SIZE) * multiplier

print(f"\n📊 TRAINING STATISTICS:")
print(f"   - Jumlah kelas: {num_classes}")
print(f"   - Images per class: ~{original_per_class}")
print(f"   - Training samples: {train_generator.samples}")
print(f"   - Multiplier: {multiplier}x")
print(f"   - Steps per epoch: {steps_per_epoch}")
print(f"   - Time per epoch: ~4 minutes")
print(f"\n🎯 Expected: 95-99% accuracy")
print(f"⏱️  Total training time: ~6-8 hours")

In [ ]:
# ============================================================
# 🏗️ BUILD MODEL
# ============================================================
print("\n" + "="*70)
print("🏗️  BUILDING MODEL - EfficientNetB4")
print("="*70 + "\n")

base_model = EfficientNetB4(
    include_top=False,
    weights='imagenet',
    input_shape=(*IMG_SIZE, 3)
)

# Freeze base model dulu
base_model.trainable = False

# Build model
inputs = base_model.input
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
outputs = Dense(num_classes, activation='softmax')(x)

model = Model(inputs, outputs)

print(f"✅ Model created!")
print(f"   Total parameters: {model.count_params():,}")
print(f"   Trainable parameters: {sum([tf.size(w).numpy() for w in model.trainable_weights]):,}")
print(f"   Non-trainable parameters: {sum([tf.size(w).numpy() for w in model.non_trainable_weights]):,}")
print("\n" + "="*70)

In [ ]:
# ============================================================
# ⚙️ COMPILE MODEL - PHASE 1
# ============================================================
model.compile(
    optimizer=AdamW(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("✅ Model compiled!")
print("   Optimizer: AdamW")
print("   Learning rate: 1e-3 (Phase 1)")
print("   Loss: categorical_crossentropy")

In [ ]:
# ============================================================
# 📋 CALLBACKS
# ============================================================
callbacks = [
    ModelCheckpoint(
        'best_model_batik.keras',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    ),
    EarlyStopping(
        monitor='val_accuracy',
        patience=20,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=7,
        min_lr=1e-7,
        verbose=1
    )
]

print("✅ Callbacks configured:")
print("   - ModelCheckpoint: Save best model")
print("   - EarlyStopping: Stop if no improvement (patience=20)")
print("   - ReduceLROnPlateau: Reduce LR if plateau (patience=7)")

In [ ]:
# ============================================================
# 🔥 PHASE 1: TRAIN TOP LAYERS
# ============================================================
print("\n" + "="*70)
print("📈 PHASE 1: Training Top Layers (Base Frozen)")
print("="*70)
print(f"\n⏱️  Estimated time: ~3-4 hours")
print(f"📊 Epochs: {EPOCHS_PHASE1}")
print(f"🎯 Target: Learn general patterns\n")
print("="*70 + "\n")

history1 = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=EPOCHS_PHASE1,
    validation_data=val_generator,
    callbacks=callbacks,
    verbose=1
)

print("\n" + "="*70)
print("✅ PHASE 1 COMPLETE!")
print("="*70)

In [ ]:
# ============================================================
# 🔓 UNFREEZE BASE MODEL FOR PHASE 2
# ============================================================
print("\n" + "="*70)
print("🔓 UNFREEZING BASE MODEL")
print("="*70 + "\n")

# Unfreeze base model
for layer in model.layers:
    if isinstance(layer, tf.keras.Model) and 'efficientnet' in layer.name.lower():
        layer.trainable = True
        print(f"✅ Unfrozen: {layer.name}")
        break

# Recompile with lower learning rate
model.compile(
    optimizer=AdamW(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print(f"\n✅ Model recompiled!")
print(f"   Learning rate: 1e-4 (10x lower for fine-tuning)")
print(f"   Trainable parameters: {sum([tf.size(w).numpy() for w in model.trainable_weights]):,}")
print("\n" + "="*70)

In [ ]:
# ============================================================
# 🚀 PHASE 2: FINE-TUNE ALL LAYERS
# ============================================================
print("\n" + "="*70)
print("🔥 PHASE 2: Fine-Tuning All Layers")
print("="*70)
print(f"\n⏱️  Estimated time: ~3-4 hours")
print(f"📊 Epochs: {EPOCHS_PHASE2}")
print(f"🎯 Target: Achieve 95-99% accuracy\n")
print("="*70 + "\n")

history2 = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=EPOCHS_PHASE1 + EPOCHS_PHASE2,
    initial_epoch=EPOCHS_PHASE1,
    validation_data=val_generator,
    callbacks=callbacks,
    verbose=1
)

print("\n" + "="*70)
print("✅ PHASE 2 COMPLETE!")
print("="*70)

In [ ]:
# ============================================================
# 💾 SAVE FINAL MODEL
# ============================================================
print("\n" + "="*70)
print("💾 SAVING MODELS")
print("="*70 + "\n")

# Save final model
model.save('final_model_batik.keras')
model.save('final_model_batik.h5')
print("✅ Saved: final_model_batik.keras")
print("✅ Saved: final_model_batik.h5")

# Save class names
import json
with open('class_names.json', 'w') as f:
    json.dump(list(train_generator.class_indices.keys()), f, indent=2)
print("✅ Saved: class_names.json")

print("\n" + "="*70)
print("✅ ALL MODELS SAVED!")
print("="*70)

In [ ]:
# ============================================================
# 📊 EVALUATE FINAL ACCURACY
# ============================================================
print("\n" + "="*70)
print("🎯 FINAL EVALUATION")
print("="*70 + "\n")

final_loss, final_acc = model.evaluate(val_generator, verbose=0)

print(f"📊 Final Results:")
print(f"   Validation Loss: {final_loss:.4f}")
print(f"   Validation Accuracy: {final_acc*100:.2f}%")

if final_acc >= 0.99:
    print("\n🎉🎉🎉 CONGRATULATIONS! Accuracy 99%+ ACHIEVED!")
    print("   PERFECT MODEL! Production ready! 🚀🚀🚀")
elif final_acc >= 0.95:
    print("\n🎉🎉 EXCELLENT! Accuracy 95%+!")
    print("   Model sangat baik untuk production! 🚀🚀")
elif final_acc >= 0.90:
    print("\n🎉 GREAT! Accuracy 90%+!")
    print("   Model bagus! 🚀")
else:
    print("\n💪 Keep training or adjust parameters for better accuracy!")

print("\n" + "="*70)

In [ ]:
# ============================================================
# 📈 PLOT TRAINING HISTORY
# ============================================================
print("\n📈 Plotting training history...\n")

plt.figure(figsize=(14, 5))

# Combine histories
acc = history1.history['accuracy'] + history2.history['accuracy']
val_acc = history1.history['val_accuracy'] + history2.history['val_accuracy']
loss = history1.history['loss'] + history2.history['loss']
val_loss = history1.history['val_loss'] + history2.history['val_loss']

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(acc, label='Train Accuracy', linewidth=2)
plt.plot(val_acc, label='Validation Accuracy', linewidth=2)
plt.axvline(x=EPOCHS_PHASE1, color='r', linestyle='--', label='Phase 2 Start')
plt.title('Model Accuracy', fontsize=14, fontweight='bold')
plt.ylabel('Accuracy', fontsize=12)
plt.xlabel('Epoch', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(loss, label='Train Loss', linewidth=2)
plt.plot(val_loss, label='Validation Loss', linewidth=2)
plt.axvline(x=EPOCHS_PHASE1, color='r', linestyle='--', label='Phase 2 Start')
plt.title('Model Loss', fontsize=14, fontweight='bold')
plt.ylabel('Loss', fontsize=12)
plt.xlabel('Epoch', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_history.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Saved: training_history.png")

In [ ]:
# ============================================================
# 📥 LIST OUTPUT FILES
# ============================================================
print("\n" + "="*70)
print("📥 OUTPUT FILES READY FOR DOWNLOAD")
print("="*70 + "\n")

output_files = [
    'best_model_batik.keras',
    'final_model_batik.keras',
    'final_model_batik.h5',
    'class_names.json',
    'training_history.png'
]

print("📦 Files generated:")
for file in output_files:
    if os.path.exists(file):
        size = os.path.getsize(file) / (1024 * 1024)  # MB
        print(f"   ✅ {file} ({size:.1f} MB)")
    else:
        print(f"   ❌ {file} (not found)")

print("\n💡 To download:")
print("   1. Click on 'Output' tab on the right")
print("   2. Click download icon next to each file")
print("   3. Or download all at once after notebook completes")

print("\n" + "="*70)
print("🎉 TRAINING COMPLETE! MODEL READY FOR PRODUCTION! 🚀")
print("="*70)